In [9]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import pyspiel
import open_spiel.python.games
from open_spiel.python.observation import make_observation
from open_spiel.python.examples.ubc_decorators import TakeSingleActionDecorator
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
from open_spiel.python.examples.ppo_utils import EnvParams
from open_spiel.python.examples.ppo_eval import eval_agents_parallel
from open_spiel.python.env_decorator import *

from auctions.webutils import *
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
import numpy as np 
import pandas as pd
from absl import logging
import tempfile
import subprocess

import open_spiel.python.games
from open_spiel.python.games.clock_auction_base import *
from open_spiel.python.examples.ubc_utils import *

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from django.db.models import F
from auctions.models import *


os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import itertools
from open_spiel.python.examples.ubc_cma import *
from open_spiel.python.examples.env_and_policy import *
from open_spiel.python.examples.ppo_eval import run_eval
from open_spiel.python.examples.straightforward_agent import StraightforwardAgent
import copy
import sys
sys.path.append('/apps/sats/python')
from pysats import map_generators, run_sats

from auctions.webutils import convert_pesky_np

from open_spiel.python.algorithms import get_all_states_with_policy
import time
import tempfile
import copy
import pickle

from open_spiel.python.algorithms.outcome_sampling_mccfr import OutcomeSamplingSolver
from open_spiel.python.algorithms.external_sampling_mccfr import ExternalSamplingSolver
import signal


In [17]:
PREFIX = 'jun19'
CONFIG_DIR = os.environ['CLOCK_AUCTION_CONFIG_DIR']
PYSATS = '/apps/sats/python'

In [18]:
with open('/apps/open_spiel/open_spiel/python/examples/oq2.pkl', 'rb') as f:
    configs = pickle.load(f)
configs

[{'scale': 1000000,
  'auction_params': {'increment': 0.3,
   'fold_randomness': True,
   'max_rounds': 5,
   'agent_memory': 5,
   'licenses': [1, 4],
   'opening_price': [32, 19],
   'activity': [32, 19]},
  'bidders': [{'player': 0,
    'types': [{'type': 'national',
      'value_per_subscriber': {'lower': 25, 'upper': 35},
      'market_share': {'lower': 0.4, 'upper': 1.1}},
     {'type': 'local',
      'value_per_subscriber': {'lower': 25, 'upper': 35},
      'market_share': {'lower': 0.25, 'upper': 0.6},
      'local_regions': [0, 1]}]},
   {'player': 1,
    'types': [{'type': 'national',
      'value_per_subscriber': {'lower': 25, 'upper': 35},
      'market_share': {'lower': 0.4, 'upper': 1.1}},
     {'type': 'local',
      'value_per_subscriber': {'lower': 25, 'upper': 35},
      'market_share': {'lower': 0.25, 'upper': 0.6},
      'local_regions': [1, 0]}]}],
  'sats_seed': 579667101,
  'map': 'OntarioQuebec'},
 {'scale': 1000000,
  'auction_params': {'increment': 0.3,
   'fo

In [19]:
# Generate a mod for each game
all_mods = defaultdict()
for i, settings in enumerate(configs):
    mods = defaultdict()
    
    ####
    settings['auction_params']['agent_memory'] = 10
    settings['auction_params']['max_rounds'] = 10
    ####
    
    mods['base'] = settings

    # Mod 1: Change to hide demand
    # hide_demand = copy.deepcopy(settings)
    # hide_demand['auction_params']['information_policy'] = 'hide_demand'
    # mods['hide_demand'] = hide_demand

    # Mod 2: Change clock speed
    high_speed = copy.deepcopy(settings)
    high_speed['auction_params']['increment'] *= 2
    mods['high_speed'] = high_speed
    
    # Mod 3: Change clock speed again
    medium_speed = copy.deepcopy(settings)
    medium_speed['auction_params']['increment'] = float(np.round(1.5 * medium_speed['auction_params']['increment'], 2))
    mods['medium_speed'] = medium_speed
    
    # Mod 4: Sanity check: allow undersell
    undersell_allowed = copy.deepcopy(settings)
    undersell_allowed['auction_params']['undersell_policy'] = UndersellPolicy.UNDERSELL_ALLOWED.name
    mods['undersell_allowed'] = undersell_allowed

    # Mod 5: Sanity check: activity doesn't matter
    no_activity = copy.deepcopy(settings)
    no_activity['auction_params']['activity_policy'] = ActivityPolicy.OFF.name
    mods['no_activity'] = no_activity
    
    # # Mod 3 imperfect base
    # imperfect = copy.deepcopy(settings)
    # imperfect['auction_params']['agent_memory'] = 2
    # mods['imperfect'] = imperfect

    all_mods[i] = mods

In [20]:
OUTPUT_DIR = f'{PYSATS}/{PREFIX}'
!mkdir -p {OUTPUT_DIR}
!mkdir -p {CONFIG_DIR}/{PREFIX}

In [21]:
# Write out YML files for the modified games
for base_name, game_mods in tqdm(all_mods.items()):
    for mod_name, mod in game_mods.items():
        g_name = f'{base_name}_{mod_name}'
        outfile_name = f'{OUTPUT_DIR}/{g_name}.yml'
        with open(outfile_name, 'w') as f:
            yaml.dump(convert_pesky_np(mod), f)
        # Run SATS
        command = f'{PYSATS}/pysats.py --config_file {outfile_name} --seed {mod["sats_seed"]} --output_file {CONFIG_DIR}/{PREFIX}/{PREFIX}_{g_name}.json'
        !python {command}

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.44s/it]
